# Practice Exam 12 - Neural Network (PyTorch) - small MLP
โจทย์: สร้าง MLP ด้วย PyTorch เพื่อทำนาย binary label จาก synthetic features
- Build a small network (2 hidden layers), train for few epochs, report accuracy
- Note: Requires PyTorch installed

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
X, y = make_classification(n_samples=500, n_features=20, n_informative=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Convert to tensors
Xtr = torch.tensor(X_train, dtype=torch.float32)
Ytr = torch.tensor(y_train, dtype=torch.long)
Xte = torch.tensor(X_test, dtype=torch.float32)
Yte = torch.tensor(y_test, dtype=torch.long)

class SimpleMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
    def forward(self, x):
        return self.net(x)

model = SimpleMLP(Xtr.shape[1])
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    logits = model(Xtr)
    loss = loss_fn(logits, Ytr)
    opt.zero_grad()
    loss.backward()
    opt.step()
    if (epoch+1) % 5 == 0:
        print('Epoch', epoch+1, 'loss', loss.item())

model.eval()
with torch.no_grad():
    preds = model(Xte).argmax(dim=1).numpy()
print('Test acc:', accuracy_score(y_test, preds))